In [1]:
# pip install seaborn

In [2]:
import pandas as pd
import numpy as np
import time
import re
from sklearn.impute import SimpleImputer
from sklearn.datasets import load_iris
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from sklearn.tree import plot_tree
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
import matplotlib.pyplot as plt

In [3]:
df = pd.read_csv("new_unicorn_data_updated.csv")

In [4]:
df.head()

,Company,Date Joined,Industry,Country,City,Continent,Year Founded,Funding,Funding_Numeric ($B),Valuation (March) ($B),Select Investors (March),Valuation (November) ($B),Select Investors (November),Years to Unicorn,Most Recent Valuation
0,SpaceX,2012-12-01,Other,United States,Hawthorne,North America,2002,$7B,7.000,100,"Founders Fund, Draper Fisher Jurvetson, Rothen...",127.0,"Founders Fund, Draper Fisher Jurvetson, Rothen...",10,127.0
1,SHEIN,2018-07-03,E-commerce And Direct-to-consumer,China,Shenzhen,Asia,2008,$2B,2.000,100,"Tiger Global Management, Sequoia Capital China...",100.0,"Tiger Global Management, Sequoia Capital China...",10,100.0
2,Stripe,2014-01-23,Fintech,United States,San Francisco,North America,2010,$2B,2.000,95,"Khosla Ventures, LowercaseCapital, capitalG",95.0,"Khosla Ventures, LowercaseCapital, capitalG",4,95.0
3,Klarna,2011-12-12,Fintech,Sweden,Stockholm,Europe,2005,$4B,4.000,46,"Institutional Venture Partners, Sequoia Capita...",6.7,"Institutional Venture Partners, Sequoia Capita...",6,6.7
4,Canva,2018-01-08,Internet Software And Services,Australia,Surry Hills,Oceania,2012,$572M,0.572,40,"Sequoia Capital China, Blackbird Ventures, Mat...",40.0,"Sequoia Capital China, Blackbird Ventures, Mat...",6,40.0


In [5]:
# df[df["Company"] == "ByteDance"]

In [6]:
df['Industry']

0                                   Other
1       E-commerce And Direct-to-consumer
2                                 Fintech
3                                 Fintech
4          Internet Software And Services
                      ...                
1034    E-commerce And Direct-to-consumer
1035    E-commerce And Direct-to-consumer
1036                  Consumer And Retail
1037                              Fintech
1038    E-commerce And Direct-to-consumer
Name: Industry, Length: 1039, dtype: object

In [7]:
# Select features and target variable
features_classification = ['Industry', 'Country']
features_regression = ['Year Founded', 'Funding_Numeric ($B)']
target_variable = 'Years to Unicorn'

In [16]:
# Split the data into training and testing sets
X_class, X_reg, y = df[features_classification], df[features_regression], df[target_variable]
X_class_train, X_class_test, X_reg_train, X_reg_test, y_train, y_test = train_test_split(
    X_class, X_reg, y, test_size=0.2, random_state=42
)

In [26]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Create a ColumnTransformer to apply OneHotEncoder only to specified columns
preprocessor = ColumnTransformer(
    transformers=[
        ('encoder', OneHotEncoder(sparse=False, drop='first'), features_classification)
    ],
    remainder='passthrough'
)

# Create a pipeline for the classifier
classifier = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', DecisionTreeClassifier())
])

# Fit the classifier
classifier.fit(X_class_train, y_train)

# Make predictions using the trained classifier
industry_country_predictions = classifier.predict(X_class_test)

# Decision Tree Regressor for Year Founded and Funding_Numeric ($B)
regressor = DecisionTreeRegressor()
regressor.fit(X_reg_train, y_train)

# Make predictions using the trained regressor
year_funding_predictions = regressor.predict(X_reg_test)

/Users/cristovalneosasono/FoDS_FinalProject/venv/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [29]:
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import GradientBoostingRegressor

# Use the predictions of the classifier as additional feature for the regressor
X_reg_train_combined = np.column_stack((X_reg_train, classifier.predict(X_class_train).reshape(-1, 1)))
X_reg_test_combined = np.column_stack((X_reg_test, classifier.predict(X_class_test).reshape(-1, 1)))

# Train a regressor (you can use a different regressor if needed)
regressor_combined = GradientBoostingRegressor()
regressor_combined.fit(X_reg_train_combined, y_train)

# Make predictions using the combined model
combined_predictions = regressor_combined.predict(X_reg_test_combined)

# Evaluate the performance
mse_combined = mean_squared_error(y_test, combined_predictions)
print(f'Mean Squared Error (Combined Model): {mse_combined}')


Mean Squared Error (Combined Model): 3.385354162515137


In [30]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error

# Use the predictions of the classifier as an additional feature for the regressor
X_reg_train_combined = np.column_stack((X_reg_train, classifier.predict(X_class_train)))
X_reg_test_combined = np.column_stack((X_reg_test, classifier.predict(X_class_test)))

# Train a Decision Tree Regressor for the combined model
combined_regressor = DecisionTreeRegressor()
combined_regressor.fit(X_reg_train_combined, y_train)

# Make predictions using the combined model
combined_predictions = combined_regressor.predict(X_reg_test_combined)

# Evaluate the performance
mse_combined = mean_squared_error(y_test, combined_predictions)
print(f'Mean Squared Error (Combined Model): {mse_combined}')


Mean Squared Error (Combined Model): 6.1311077315541604
